In [1]:
import requests
import json
import prettytable
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import io

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
file_metro = "data_files\MSA_M2020_dl.xlsx"
bls_metro_df = pd.read_excel(file_metro)
bls_metro_df.head()

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,O_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC_QUOTIENT,PCT_TOTAL,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,total,66060,1.9,1000,1,NaN,20.64,42930,1.8,9.13,11.4,16.41,24.82,36.23,18990,23710,34130,51620,75370,NaN,NaN
1,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,major,2910,4.5,44.068,0.77,NaN,42.87,89160,2.2,17.38,26.74,36.37,51.68,73.88,36150,55620,75640,107500,153670,NaN,NaN
2,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-1021,General and Operations Managers,detailed,1320,7.4,20.036,1.19,NaN,40.38,83990,3.5,12.67,24.02,32.96,50.11,72.93,26350,49960,68550,104230,151700,NaN,NaN
3,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-2022,Sales Managers,detailed,90,18.5,1.317,0.47,NaN,58.18,121020,7.6,26.23,35.9,52.41,68.01,#,54560,74680,109000,141460,#,NaN,NaN
4,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-2030,Public Relations and Fundraising Managers,detailed,40,31.5,0.549,0.94,NaN,45.93,95540,20.7,28.71,32.78,36.24,41.23,83.81,59710,68180,75380,85760,174320,NaN,NaN


In [26]:
bls_metro_df.isnull().sum()

AREA                 0
AREA_TITLE           0
AREA_TYPE            0
PRIM_STATE           0
NAICS                0
NAICS_TITLE          0
I_GROUP              0
OWN_CODE             0
OCC_CODE             0
OCC_TITLE            0
O_GROUP              0
TOT_EMP              0
EMP_PRSE             0
JOBS_1000            0
LOC_QUOTIENT         0
PCT_TOTAL       140051
H_MEAN               0
A_MEAN               0
MEAN_PRSE            0
H_PCT10              0
H_PCT25              0
H_MEDIAN             0
H_PCT75              0
H_PCT90              0
A_PCT10              0
A_PCT25              0
A_MEDIAN             0
A_PCT75              0
A_PCT90              0
ANNUAL          132852
HOURLY          139836
dtype: int64

In [4]:
bls_metro_df.nunique()

AREA              396
AREA_TITLE        396
AREA_TYPE           1
PRIM_STATE         52
NAICS               1
NAICS_TITLE         1
I_GROUP             1
OWN_CODE            1
OCC_CODE          808
OCC_TITLE         808
O_GROUP             3
TOT_EMP          3808
EMP_PRSE          501
JOBS_1000       21773
LOC_QUOTIENT     1512
PCT_TOTAL           0
H_MEAN           7982
A_MEAN          14144
MEAN_PRSE         297
H_PCT10          4987
H_PCT25          5969
H_MEDIAN         6814
H_PCT75          7957
H_PCT90          8740
A_PCT10          8810
A_PCT25         10636
A_MEDIAN        12570
A_PCT75         15067
A_PCT90         16941
ANNUAL              1
HOURLY              1
dtype: int64

In [5]:
bls_50metro_df = bls_metro_df.drop(['AREA_TYPE',
                                    'NAICS',
                                    'NAICS_TITLE',
                                    'I_GROUP',
                                    'OWN_CODE',
                                    'PCT_TOTAL',
                                    'ANNUAL',
                                    'HOURLY'] ,axis=1)
bls_50metro_df.head()

,AREA,AREA_TITLE,PRIM_STATE,OCC_CODE,OCC_TITLE,O_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC_QUOTIENT,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
0,10180,"Abilene, TX",TX,00-0000,All Occupations,total,66060,1.9,1000,1,20.64,42930,1.8,9.13,11.4,16.41,24.82,36.23,18990,23710,34130,51620,75370
1,10180,"Abilene, TX",TX,11-0000,Management Occupations,major,2910,4.5,44.068,0.77,42.87,89160,2.2,17.38,26.74,36.37,51.68,73.88,36150,55620,75640,107500,153670
2,10180,"Abilene, TX",TX,11-1021,General and Operations Managers,detailed,1320,7.4,20.036,1.19,40.38,83990,3.5,12.67,24.02,32.96,50.11,72.93,26350,49960,68550,104230,151700
3,10180,"Abilene, TX",TX,11-2022,Sales Managers,detailed,90,18.5,1.317,0.47,58.18,121020,7.6,26.23,35.9,52.41,68.01,#,54560,74680,109000,141460,#
4,10180,"Abilene, TX",TX,11-2030,Public Relations and Fundraising Managers,detailed,40,31.5,0.549,0.94,45.93,95540,20.7,28.71,32.78,36.24,41.23,83.81,59710,68180,75380,85760,174320


In [6]:
x = bls_50metro_df.columns.to_list()
x

['AREA',
 'AREA_TITLE',
 'PRIM_STATE',
 'OCC_CODE',
 'OCC_TITLE',
 'O_GROUP',
 'TOT_EMP',
 'EMP_PRSE',
 'JOBS_1000',
 'LOC_QUOTIENT',
 'H_MEAN',
 'A_MEAN',
 'MEAN_PRSE',
 'H_PCT10',
 'H_PCT25',
 'H_MEDIAN',
 'H_PCT75',
 'H_PCT90',
 'A_PCT10',
 'A_PCT25',
 'A_MEDIAN',
 'A_PCT75',
 'A_PCT90']

In [7]:
x = [col.lower() for col in x]
x

['area',
 'area_title',
 'prim_state',
 'occ_code',
 'occ_title',
 'o_group',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc_quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90']

In [8]:
bls_50metro_df.columns = x

In [9]:
bls_50metro_df.head()

,area,area_title,prim_state,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,10180,"Abilene, TX",TX,00-0000,All Occupations,total,66060,1.9,1000,1,20.64,42930,1.8,9.13,11.4,16.41,24.82,36.23,18990,23710,34130,51620,75370
1,10180,"Abilene, TX",TX,11-0000,Management Occupations,major,2910,4.5,44.068,0.77,42.87,89160,2.2,17.38,26.74,36.37,51.68,73.88,36150,55620,75640,107500,153670
2,10180,"Abilene, TX",TX,11-1021,General and Operations Managers,detailed,1320,7.4,20.036,1.19,40.38,83990,3.5,12.67,24.02,32.96,50.11,72.93,26350,49960,68550,104230,151700
3,10180,"Abilene, TX",TX,11-2022,Sales Managers,detailed,90,18.5,1.317,0.47,58.18,121020,7.6,26.23,35.9,52.41,68.01,#,54560,74680,109000,141460,#
4,10180,"Abilene, TX",TX,11-2030,Public Relations and Fundraising Managers,detailed,40,31.5,0.549,0.94,45.93,95540,20.7,28.71,32.78,36.24,41.23,83.81,59710,68180,75380,85760,174320


In [10]:
x

['area',
 'area_title',
 'prim_state',
 'occ_code',
 'occ_title',
 'o_group',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc_quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90']

In [11]:
cols = ['tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc_quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90']

for col in cols:
    bls_50metro_df[col] = pd.to_numeric(bls_50metro_df[col], errors='coerce')

bls_50metro_df.head()

,area,area_title,prim_state,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,10180,"Abilene, TX",TX,00-0000,All Occupations,total,66060.0,1.9,1000.000,1.00,20.64,42930.0,1.8,9.13,11.40,16.41,24.82,36.23,18990.0,23710.0,34130.0,51620.0,75370.0
1,10180,"Abilene, TX",TX,11-0000,Management Occupations,major,2910.0,4.5,44.068,0.77,42.87,89160.0,2.2,17.38,26.74,36.37,51.68,73.88,36150.0,55620.0,75640.0,107500.0,153670.0
2,10180,"Abilene, TX",TX,11-1021,General and Operations Managers,detailed,1320.0,7.4,20.036,1.19,40.38,83990.0,3.5,12.67,24.02,32.96,50.11,72.93,26350.0,49960.0,68550.0,104230.0,151700.0
3,10180,"Abilene, TX",TX,11-2022,Sales Managers,detailed,90.0,18.5,1.317,0.47,58.18,121020.0,7.6,26.23,35.90,52.41,68.01,NaN,54560.0,74680.0,109000.0,141460.0,NaN
4,10180,"Abilene, TX",TX,11-2030,Public Relations and Fundraising Managers,detailed,40.0,31.5,0.549,0.94,45.93,95540.0,20.7,28.71,32.78,36.24,41.23,83.81,59710.0,68180.0,75380.0,85760.0,174320.0


In [12]:
bls_50metro_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140051 entries, 0 to 140050
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          140051 non-null  int64  
 1   area_title    140051 non-null  object 
 2   prim_state    140051 non-null  object 
 3   occ_code      140051 non-null  object 
 4   occ_title     140051 non-null  object 
 5   o_group       140051 non-null  object 
 6   tot_emp       128393 non-null  float64
 7   emp_prse      128393 non-null  float64
 8   jobs_1000     128393 non-null  float64
 9   loc_quotient  128393 non-null  float64
 10  h_mean        130767 non-null  float64
 11  a_mean        137203 non-null  float64
 12  mean_prse     137075 non-null  float64
 13  h_pct10       130746 non-null  float64
 14  h_pct25       130508 non-null  float64
 15  h_median      129899 non-null  float64
 16  h_pct75       129119 non-null  float64
 17  h_pct90       126905 non-null  float64
 18  a_pc

In [13]:
bls_50metro_df = bls_50metro_df.dropna()

In [28]:
bls_50metro_df.isnull().sum()

area            0
area_title      0
prim_state      0
occ_code        0
occ_title       0
o_group         0
tot_emp         0
emp_prse        0
jobs_1000       0
loc_quotient    0
h_mean          0
a_mean          0
mean_prse       0
h_pct10         0
h_pct25         0
h_median        0
h_pct75         0
h_pct90         0
a_pct10         0
a_pct25         0
a_median        0
a_pct75         0
a_pct90         0
dtype: int64

In [14]:
bls_50metro_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115629 entries, 0 to 140049
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          115629 non-null  int64  
 1   area_title    115629 non-null  object 
 2   prim_state    115629 non-null  object 
 3   occ_code      115629 non-null  object 
 4   occ_title     115629 non-null  object 
 5   o_group       115629 non-null  object 
 6   tot_emp       115629 non-null  float64
 7   emp_prse      115629 non-null  float64
 8   jobs_1000     115629 non-null  float64
 9   loc_quotient  115629 non-null  float64
 10  h_mean        115629 non-null  float64
 11  a_mean        115629 non-null  float64
 12  mean_prse     115629 non-null  float64
 13  h_pct10       115629 non-null  float64
 14  h_pct25       115629 non-null  float64
 15  h_median      115629 non-null  float64
 16  h_pct75       115629 non-null  float64
 17  h_pct90       115629 non-null  float64
 18  a_pc

In [15]:
#create CSV files to be used to load database as needed
bls_50metro_df.to_csv('data_files/bls_50metro_2020_clean.csv')


Remove the comment to have dataFrames load to postGres DB

In [30]:
from config import AWS

In [32]:
#connecting to postgres DB to create a new table
engine = create_engine(AWS)

bls_50metro_df.to_sql('bls50metro2020', engine, if_exists='replace')

